#### Procurement & Logistic Logbook - Automated Processing Pipeline

##### 1. Overview
This document outlines the automated data processing pipeline for the Procurement & Logistic Logbook. The purpose of this script is to consolidate a complex, multi-stage process from five separate notebooks into a single, efficient, and maintainable Python script.

The script reads the main PO Entry List and several supplementary lookup files, performs a series of transformations and calculations to generate over 50 new feature columns, and outputs a single, clean Excel file ready for analysis or reporting.

#### 2. Prerequisites & Setup
Before running the script, ensure the following requirements are met.

Required Libraries
The script requires the following Python libraries. You can install them using pip:

In [1]:
### 1. SETUP AND IMPORTS
import pandas as pd
import numpy as np
import os
import math
import datetime as dt
import re

In [2]:
### 2. FUNCTIONS
def LOC_strings(s):
    """
    Cleans and standardizes the location string from the 'Department' column,
    correctly identifying HO vs LC based on underscores.
    """
    if pd.isna(s):
        return 'Unknown'
        
    # Remove the 'x' at the start if present, and strip any extra whitespace
    s = s[1:].strip() if s.startswith('x') else s.strip()

    # Convert to uppercase to ensure case-insensitive matching
    s = s.upper()

    # Broad check for simple HO cases first
    if s.endswith('-HO'):
        return "HO"

    # Initialize loc_type and location
    loc_type = 'Unknown'
    location = ''

    # --- CORRECTED LOGIC to determine location type ---
    if '__' in s:
        loc_type = 'HO'
    elif '_' in s:
        loc_type = 'LC'
    else:
        # Default to 'LC' if a location is found but no loc_type indicator
        loc_type = 'LC'

    # Determine the specific location name
    if 'PALU' in s: location = 'PALU'
    elif 'FLUK' in s: location = 'FLUK'
    elif 'LAR' in s: location = 'LAR'
    elif 'LWK' in s: location = 'LWK'
    elif 'OBI' in s: location = 'OBI'
    elif 'KDI' in s: location = 'KDI'
    elif 'BARU' in s: location = 'BARU'
    elif 'MUNA' in s: location = 'MUNA'
    elif 'LWI' in s: location = 'LWI'
    elif 'POM' in s: location = 'POM'
    elif 'KNW' in s: location = 'KNW'
    elif 'WATU' in s: location = 'WATU'
    elif 'LAEYA' in s: location = 'LAEYA'

    # If a HO-type location is found, combine them
    if loc_type == 'HO' and location:
        return f"HO {location}".strip()
    
    return f"{loc_type} {location}".strip()

def project_string(s):
    """Extracts the project name from the 'Department' column."""
    if pd.isna(s):
        return None
    if s.startswith('x'):
        s = s[1:].strip()
    
    first_word = s.split('-')[0].split('_')[0]

    if first_word == "IMS":
        if "147" in s: return "IMS 147"
        if "52" in s: return "IMS 52"
        return "IMS"
    
    if first_word == "MPS":
        if "SC" in s: return "MPS SC"
        return "MPS"

    if first_word == "MMP":
        if "SC" in s: return "MMP SC"
        if s.endswith('HO'): return f"{first_word} HO"
        if s.endswith('LAR'): return f"{first_word} LAR"
        if s.endswith('WATU'): return f"{first_word} WATU"
        if s.endswith('OBI'): return f"{first_word} OBI"
        if s.endswith('POM'): return f"{first_word} POM"
        if s.endswith('LAEYA'): return f"{first_word} LAEYA"
        # Note: KDI mapped into LAR
        if s.endswith('KDI'): return f"{first_word} LAR"
        return first_word
    return first_word

def divisi_string(s):
    """Extracts the division from the 'Department' column."""
    if pd.isna(s):
        return None
    parts = s.split('-')
    if len(parts) > 1:
        sub_parts = parts[1].split('_')
        first_part = sub_parts[0]
        if "HRGA" in first_part:
            return "HRGA"
        return first_part
    return None

def PTCV_strings(s):
    """Categorizes supplier as PT, CV, or Other."""
    if pd.isna(s):
        return 'Toko/Lainnya'
    if s.startswith('PT'): return 'PT'
    if s.startswith('CV'): return 'CV'
    return 'Toko/Lainnya'

def TOP_strings(s):
    """Categorizes Term of Payment."""
    if pd.isna(s): return 'Empty'
    s = str(s).lower().strip()
    
    progressive_keywords = ['dp', 'downpayment', 'down payment', 'pembayaran 1 ', '50% sebelum', 'kredit', 'tahap', 'leasing', 'installment']
    tbd_keywords = ['pembayaran sebelum', 'before delivery', 'sebelum pengiriman', 'cash', 'uang muka', '100% sebelum', 'tunai', 'seelum', 'sebeulm', 'transfer', 'pengiriman setelah pembayaran', 'setelah pembayaran']
    tempo_keywords = ['hari setelah', 'hari dari', 'tempo', 'invoice diterima', 'penagihan dilakukan', 'setelah pengiriman', 'pembayaran setelah', 'kontrak', 'after delivery', 'hari kerja', 'telah diterima', 'pembayaran per bulan', 'ari', '0', 'pekerjaan pengujian dilakukan setelah pembayaran dilakukan', '100% di muka']
    
    if any(keyword in s for keyword in progressive_keywords): return 'Progressive'
    if any(keyword in s for keyword in tbd_keywords): return 'TBD'
    if any(keyword in s for keyword in tempo_keywords): return 'Tempo'
    return 'Not Applicable'

def item_category_merged(item_category, unit=None):
    """Merges similar item categories, including Tire DT handling."""
    if not isinstance(item_category, str):
        return item_category  # return unchanged if not string
    
    item_category_upper = item_category.upper().strip()
    unit_upper = str(unit).upper().strip() if isinstance(unit, str) else ""

    if 'XCMG' in item_category_upper:
        return 'Spare Part XCMG'
    if 'SANY' in item_category_upper:
        return 'Spare Part SANY'
    if 'ZS' in item_category_upper:
        return 'Spare Part ZS'

    if 'TIRE DT' in item_category_upper:
        return 'Tire DT - Set' if 'SET' in unit_upper else 'Tire DT - non Set'

    return item_category

def category_value_marker(row):
    """Marks categories that should not be counted for performance."""
    specific_categories = ["Kontrak", "Seragam", "Jasa Logistik", "Jasa/Service", "ATK", "Cetak", "Makanan dan Minuman", "Seragam Security", "x Kebutuhan Kantin", "x Kebutuhan Mess", "x Medical dan Obat"]
    item_category = str(row['Item Category']).strip()   
    requisition_type = str(row['Requisition Type']).strip()
    item_name = str(row['Item Name']).strip().lower()
    
    if item_category in specific_categories or requisition_type == "Consignment" or (item_category == "APD" and "sepatu" in item_name):
        return 1
    return 0

def category_value_xcmg(row):
    """Marks specific XCMG-related transactions."""
    requisition_type = row['Requisition Type']
    item_category = row['Item Category']
    background_needs = row['Background Needs']
    keywords = ["Pengambilan", "Berita acara pengeluaran", "BA", "Consignment"]
    
    if (isinstance(requisition_type, str) and requisition_type != "Consignment" and
        isinstance(item_category, str) and "XCMG" in item_category and
        isinstance(background_needs, str) and any(keyword in background_needs for keyword in keywords)):
        return 1
    return 0

def urgent_normal_function(row):
    """Determines if a request is Urgent or Normal based on lead time."""
    if row['Requisition Type'] == 'Consignment': return 'Normal'
    
    loc = str(row['LOC']) if not pd.isnull(row['LOC']) else ''
    lead_time = row['LEAD TIME']
    if pd.isna(lead_time): return 'Normal'

    if (loc == 'HO' and lead_time <= 15) or \
       ('LC' in loc and lead_time <= 15) or \
       (loc in ['HO LAR', 'HO LWK', 'HO PALU', 'HO KDI', 'HO MUNA', 'HO KNW'] and lead_time <= 36) or \
       (loc in ['HO OBI', 'HO FLUK', 'HO BARU'] and lead_time <= 43):
        return 'Urgent'
    return 'Normal'

lebaran_2025 = pd.to_datetime(pd.date_range(start='2025-03-28', end='2025-04-13')).date
lebaran_set = set(lebaran_2025)

def days_excluding_lebaran(start_date, end_date):
    """
    Calculates the number of non-holiday business days between two dates.
    This version is robust and correctly handles early/late deliveries within holiday periods.
    """
    if pd.isnull(start_date) or pd.isnull(end_date):
        return np.nan

    start_d = pd.to_datetime(start_date).date()
    end_d = pd.to_datetime(end_date).date()

    if start_d == end_d:
        return 0

    is_early = end_d < start_d
    range_start = min(start_d, end_d)
    range_end = max(start_d, end_d)
    date_range = pd.date_range(start=range_start, end=range_end)[1:]
    non_lebaran_days = sum(1 for d in date_range if d.date() not in lebaran_set)

    if is_early:
        return -non_lebaran_days
    else:
        return non_lebaran_days

def determine_freight(row, freight_mapping, rara_datasheet, ryi_datasheet):
    """Determines the freight type based on a hierarchy of rules."""
    supplier = row['Supplier']
    po_number = row['PO Number']

    if not isinstance(supplier, str):
        return "Other Freight"

    if supplier in freight_mapping and pd.notnull(freight_mapping[supplier]):
        return freight_mapping[supplier]
    
    if "RARA" in supplier.upper():
        rara_match = rara_datasheet.loc[rara_datasheet['PO Number'] == po_number, 'Freight Type']
        return rara_match.iloc[0] if not rara_match.empty else "Unknown RARA Freight"

    if "RYI" in supplier.upper():
        ryi_match = ryi_datasheet.loc[ryi_datasheet['PO Number'] == po_number, 'Freight Type']
        return ryi_match.iloc[0] if not ryi_match.empty else "Unknown RYI Freight"
        
    return "Other Freight"

def extract_finalisasi_date(text):
    """
    Extracts a date (dd/mm/yyyy or dd Mmm yyyy) that immediately follows 
    'Finalisasi' (case-insensitive) in the text string using Regex.
    """
    if pd.isna(text):
        return None
    text = str(text)
    regex = r"finalisasi.*?\s*(\d{1,2}[/\s][A-Za-z0-9]{2,3}[/\s]\d{4})"
    match = re.search(regex, text, re.IGNORECASE)
    if match:
        date_string = match.group(1).strip().replace('.', '')
        try:
            return pd.to_datetime(date_string, dayfirst=True, errors='raise').date()
        except Exception:
            return None
    else:
        return None

print("All helper functions defined.")

All helper functions defined.


In [3]:
### 3. DATA LOADING
base_path = r'C:\Users\user\Documents\PROC\Monthly - Weekly\2025\11November\11November Week 2 1011'
dir_path = r'C:\Users\user\Documents\code\Weekly Process\Dir'

try:
    df = pd.read_excel(os.path.join(base_path, 'PO Entry List.xlsx'))
    picnorm = pd.read_excel(r'C:\Users\user\Documents\PROC\Monthly - Weekly\####1Normalisasi\Normalisasi.xlsx', sheet_name='RFM')
    holidays_input = pd.read_excel(os.path.join(dir_path, 'nonworkday2024.xlsx'))
    WilayahDir = pd.read_excel(os.path.join(dir_path, 'wilayah.xlsx'))
    SupplierDir = pd.read_excel(os.path.join(dir_path, 'pulau.xlsx'))
    js = pd.read_excel(os.path.join(dir_path, 'jasaservice.xlsx'))
    logistic_freight_path = r'C:\Users\User\Documents\PROC\Monthly - Weekly\Logistic Freight.xlsx'
    freight_datasheet = pd.read_excel(logistic_freight_path, sheet_name='Freight')
    rara_datasheet = pd.read_excel(logistic_freight_path, sheet_name='RARA')
    ryi_datasheet = pd.read_excel(logistic_freight_path, sheet_name='RYI')
    cost_saving_df = pd.read_excel(r'C:\Users\User\Documents\PROC\Monthly - Weekly\Cost Saving.xlsx')
    idlibrary_df = pd.read_csv(os.path.join(dir_path, 'idlibrary.csv'))
    
    # Load all normalization sheets
    ontime_norm_path = os.path.join(dir_path, 'ontime_normalisasi.xlsx')
    datanormal = pd.read_excel(ontime_norm_path, sheet_name='normalized')
    timedatenorm = pd.read_excel(ontime_norm_path, sheet_name='timedate')
    datnotcounted = pd.read_excel(ontime_norm_path, sheet_name='notcalculated')
    logistic_normalized = pd.read_excel(ontime_norm_path, sheet_name='logistic_normalized')


    print("All data loaded successfully.")
    print(f"Main dataframe shape: {df.shape}")

except FileNotFoundError as e:
    print(f"Error: A file was not found. Please check your file paths.")
    print(e)

All data loaded successfully.
Main dataframe shape: (79263, 58)


In [ ]:
### 4. DATA PROCESSING AND FEATURE ENGINEERING

print("Preparing data and normalizing dates...")
date_columns = [
    'Requisition Approved Date', 'Requisition Required Date', 'PO Submit Date',
    'PO Approval Date', 'Receive PO Date', 'Created TL Date', 'Shipped Date',
    'Received TL Date', 'PO Required Date'
]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

holidays = pd.to_datetime(holidays_input['NONWORKDAYS'], format='%d/%m/%Y').dt.date.tolist()
picnorm_indexed = picnorm.drop_duplicates(subset='Requisition Number').set_index('Requisition Number')

# --- Step 1: Initial Feature Engineering (REVISED PRIORITY) ---
print("Running Step 1: Initial Feature Engineering...")

# A. Extract date from Req Progress Status using the new function
df['Extracted Approved Date'] = df['Req Progress Status'].apply(extract_finalisasi_date)

# B. Map dates from picnorm
df['Updated Requisition Approved Date'] = df['Requisition Number'].map(picnorm_indexed.get('Updated Requisition Approved Date'))
df['Updated Requisition Required Date'] = df['Requisition Number'].map(picnorm_indexed.get('Updated Requisition Required Date'))
df['Background Update'] = df['Requisition Number'].map(picnorm_indexed.get('Background Update'))

# C. Apply Priority Logic (Highest: picnorm -> Second: Extracted -> Lowest: Original)
used_approved_date = (
    # 1. Start with the highest priority: picnorm's normalized date
    df['Updated Requisition Approved Date']
    # 2. If picnorm is NaN, use the date extracted from Req Progress Status
    .combine_first(df['Extracted Approved Date'])
    # 3. If both are NaN, use the original date
    .combine_first(df['Requisition Approved Date'])
)

used_required_date = df['Updated Requisition Required Date'].fillna(df['Requisition Required Date'])

# D. Convert final selected dates to datetime objects for calculations
used_approved_date = pd.to_datetime(used_approved_date, errors='coerce')
used_required_date = pd.to_datetime(used_required_date, errors='coerce')

df['LOC'] = df['Department'].apply(LOC_strings)
df['LEAD TIME'] = np.floor((used_required_date - used_approved_date).dt.total_seconds() / (24 * 3600))
df['URGENT_NORMAL'] = df.apply(urgent_normal_function, axis=1)

# Correctly calculate URGENT*, NORMAL, URGENT2, and URGENT_FINALFORLOGBOOK
df['URGENT*'] = df.apply(
    lambda row: 'Urgent*' if str(row['Urgent']) == 'Normal' and row['URGENT_NORMAL'] == 'Urgent' else np.nan,
    axis=1
)
df['NORMAL'] = np.where(df['URGENT_NORMAL'] == 'Normal', 'Normal', None)
df['URGENT2'] = np.where(df['URGENT_NORMAL'] == 'Urgent', 'Urgent', None)
df['URGENT_FINALFORLOGBOOK'] = df['URGENT*'].combine_first(df['URGENT_NORMAL'])

# Wilayah and Pulau
temp_df = df[['Supplier Location']].copy()
temp_df['Supplier Location'] = temp_df['Supplier Location'].str.strip().str.lower()
WilayahDir['Supplier Location'] = WilayahDir['Supplier Location'].str.strip().str.lower()
wilayah_process = pd.merge(temp_df, WilayahDir, on='Supplier Location', how='left')
SupplierDir['Wilayah'] = SupplierDir['Wilayah'].str.strip().str.lower()
wilayah_process['To'] = wilayah_process['To'].str.lower()
supplier_process = pd.merge(wilayah_process, SupplierDir, left_on='To', right_on='Wilayah', how='left')
df['WILAYAH'] = supplier_process['To']
df['PULAU'] = supplier_process['Pulau']

df['DEPARTMENT_'] = df['Department'].apply(project_string)
df['DIVISI'] = df['Department'].apply(divisi_string)
df['SUPPLIER_'] = df['Supplier'].apply(PTCV_strings)
df['TOP'] = df['Term of Payment'].apply(TOP_strings)
df['CATEGORYMERGED'] = df.apply(
    lambda x: item_category_merged(x['Item Category'], x.get('Unit')), 
    axis=1
)
df['CATEGORYVALUE'] = df.apply(category_value_marker, axis=1)
df['CATEGORYVALUEXCMG'] = df.apply(category_value_xcmg, axis=1)
df['VALUE'] = 1
df.loc[df.groupby("PO Number")["CATEGORYVALUE"].transform("max") == 1, "VALUE"] = 0
df.loc[df.groupby("PO Number")["CATEGORYVALUEXCMG"].transform("max") == 1, "VALUE"] = 0
first_occurrence_mask = ~df.duplicated(subset=['PO Number'])
df.loc[first_occurrence_mask & (df.groupby('PO Number')['VALUE'].transform('sum') == 0), 'VALUE'] = 1
df['UNIQUE COUNT PO'] = np.where(~df['PO Number'].duplicated(), 1, 0)
df['Final_ItemID'] = df['Item ID']


# --- Step 2: Time-Based Calculations ---
print("Running Step 2: Time-Based Calculations...")

# --- REVISED LOGIC FOR TIME DATE ---
sulawesi_locations = ['LAR', 'LWK', 'PALU', 'KDI', 'MUNA', 'TKE', 'WATU']
halmahera_locations = ['OBI', 'FLUK', 'BARU', 'LWI']
ho_sulawesi_pattern = '|'.join([f'HO {loc}' for loc in sulawesi_locations])
ho_halmahera_pattern = '|'.join([f'HO {loc}' for loc in halmahera_locations])
lc_sulawesi_pattern = '|'.join([f'LC {loc}' for loc in sulawesi_locations])
lc_halmahera_pattern = '|'.join([f'LC {loc}' for loc in halmahera_locations])
LC_Categorizatoin = [
    'Consumable Workshop', 'Packaging', 'Alat dan Bahan Bangunan', 
    'Bolt dan Nut', 'Elektrikal', 'Consumable Cleaning',
    'Perabotan', 'Peralatan Geologi', 'Peralatan Dapur'
]
is_special_lc_category = df['Item Category'].isin(LC_Categorizatoin)
conditions = [
    df['LOC'].str.contains(ho_halmahera_pattern, na=False),
    df['LOC'].str.contains(ho_sulawesi_pattern, na=False),
    df['LOC'].str.contains(lc_halmahera_pattern, na=False) & is_special_lc_category,
    df['LOC'].str.contains(lc_sulawesi_pattern, na=False) & is_special_lc_category,
    df['LOC'].str.contains(r"LC|HO", na=False)
]
choices = [43, 36, 43, 36, 15]
days_to_add = np.select(conditions, choices, default=0).astype(int)

# 2. Create the normalization map and apply it
timedate_norm_map = timedatenorm.set_index('PO Number')['timedate']
overrides = df['PO Number'].map(timedate_norm_map)

# 3. Coalesce the override with the original calculation
final_days_to_add = overrides.fillna(pd.Series(days_to_add, index=df.index)).astype(int)
df['TIME DATE'] = used_approved_date + pd.to_timedelta(final_days_to_add, unit='D')

is_calculable = (
    (df['VALUE'] == 1)
    & (df['Requisition Type'] != 'Consignment')
    & (~df['Item Category'].isin(['Jasa Logistik', 'Solar']))
)
is_calculable_wd = is_calculable & used_approved_date.notna() & df['PO Submit Date'].notna()
df['PR - PO'] = np.where(is_calculable, [days_excluding_lebaran(s, e) for s, e in zip(used_approved_date, df['PO Submit Date'])], np.nan)
df['PR - PO'] = np.where(df['PR - PO'] < 0, 0, df['PR - PO'])
df['PO SUB - PO APP'] = np.where(is_calculable, [days_excluding_lebaran(s, e) for s, e in zip(df['PO Submit Date'], df['PO Approval Date'])], np.nan)
df['PO - R PO'] = np.where(df['PR - PO'].notna() & df['Receive PO Date'].notna() & (df['Item Category'] != 'Jasa/Service'), [days_excluding_lebaran(s, e) for s, e in zip(df['PO Approval Date'], df['Receive PO Date'])], np.nan)
df['R-R SITE'] = np.where(df['PR - PO'].notna() & df['Receive PO Date'].notna() & (df['Item Category'] != 'Jasa/Service') & df['Location TL Received'].notna() & (df['LOC'] != 'HO'), [days_excluding_lebaran(s, e) for s, e in zip(df['Receive PO Date'], df['Received TL Date'])], np.nan)
df['PR - PO SUB WD'] = np.nan
start_dates_filtered = pd.to_datetime(used_approved_date[is_calculable_wd], errors='coerce')
end_dates_filtered = pd.to_datetime(df.loc[is_calculable_wd, 'PO Submit Date'], errors='coerce')
final_valid_mask = start_dates_filtered.notna() & end_dates_filtered.notna()
final_valid_index = start_dates_filtered[final_valid_mask].index
valid_start_dates = start_dates_filtered[final_valid_mask]
valid_end_dates = end_dates_filtered[final_valid_mask]
df.loc[final_valid_index, 'PR - PO SUB WD'] = np.busday_count(
    valid_start_dates.values.astype('datetime64[D]'),
    valid_end_dates.values.astype('datetime64[D]'),
    weekmask='1111100',
    holidays=holidays
)
df['REQUISITION_TOTAL'] = (df['Requisition SubTotal'] * df['Exchange Rate']) * 1.11
df['PO_TOTAL'] = (df['Qty Order'] * df['PO Price'] * df['Exchange Rate']) + (df['Jumlah PPN'] * df['Exchange Rate'])
df['BUDGET'] = (df['REQUISITION_TOTAL'] - df['PO_TOTAL']).round(2)
df['BUDGET%'] = (df['PO_TOTAL'] / df['REQUISITION_TOTAL']).clip(upper=1)
df['RPO-TLC'] = np.where(df['R-R SITE'].notna(), (df['Created TL Date'] - df['Receive PO Date']).dt.days, np.nan)
df['TLC-SHIP'] = np.where(df['R-R SITE'].notna(), (df['Shipped Date'] - df['Created TL Date']).dt.days, np.nan)
df['SHIP-RSITE'] = np.where(df['R-R SITE'].notna(), (df['Received TL Date'] - df['Shipped Date']).dt.days, np.nan)

# --- NEW: Purchasing On-Time Calculation ---
print("Calculating Purchasing On-Time metric...")
# --- REVISED LOGIC: Use the 'is_calculable' mask for consistency ---
df['Purchasing_Duration'] = np.where(
    is_calculable, # This mask already checks for VALUE=1 and not Consignment
    [days_excluding_lebaran(s, e) for s, e in zip(used_approved_date, df['PO Approval Date'])],
    np.nan
)
# --- END OF REVISION ---

# Define team members
HO_team = ['Linda / Puji / Syifa R / Stheven', 'Syifa Ramadhani', 'Syifa Alifia', 'Rizal Agus Fianto', 'Auriel', 'Puji Astuti', 'Linda Permata Sari', 'Laurensius Adi', 'Syifa Ramadhani Luthfi']
Site_team = ['Rona / Joko', 'Joko', 'Victo', 'Rakan', 'Rona Justhafist', 'Rona / Victo / Rakan / Joko', 'Fairus / Irwan', 'Fairus Mubakri', 'Irwan', 'Ady', 'Fairus / Ady', 'Olvan']

# Create boolean masks for each team
is_ho_team = df['Procurement Name'].isin(HO_team)
is_site_team = df['Procurement Name'].isin(Site_team)

# Set status based on a hierarchy of rules (no longer need VALUE check here)
conditions = [
    # HO Team Rules (<= 5 days)
    (is_ho_team) & (df['Purchasing_Duration'] <= 5),
    (is_ho_team) & (df['Purchasing_Duration'] > 5),
    
    # Site Team Rules (<= 3 days)
    (is_site_team) & (df['Purchasing_Duration'] <= 3),
    (is_site_team) & (df['Purchasing_Duration'] > 3)
]
choices = ['On Time', 'Late', 'On Time', 'Late']
df['STATUS_Purchasing'] = np.select(conditions, choices, default=None)

# Purchasing Flag
df['ON_TIME_Purchasing'] = np.where(df['STATUS_Purchasing'] == 'On Time', 1, np.nan)
df['LATE_Purchasing'] = np.where(df['STATUS_Purchasing'] == 'Late', 1, np.nan)
df['ON_TIME%_Purchasing'] = np.where(df['STATUS_Purchasing'] == 'On Time', 1, np.where(df['STATUS_Purchasing'] == 'Late', 0, np.nan))
# --- END OF NEW CALCULATION ---


# --- Step 3: Logistics & Receiving Status ---
print("Running Step 3: Logistics & Receiving Status...")
freight_mapping = dict(zip(freight_datasheet['Supplier'], freight_datasheet['Freight Type']))
df['LOGISTIC_FREIGHT'] = np.where(df['Item Category'] == 'Jasa Logistik', df.apply(determine_freight, axis=1, args=(freight_mapping, rara_datasheet, ryi_datasheet)), '')
df['FARTHEST REQUIRED DATE'] = df[['PO Required Date', 'Requisition Required Date', 'TIME DATE']].max(axis=1)
df['USED RECEIVE DATE'] = df['Receive PO Date'].fillna(df['Received TL Date'])
df['REC'] = np.where(df['VALUE'] == 1, [days_excluding_lebaran(s, e) for s, e in zip(df['FARTHEST REQUIRED DATE'], df['USED RECEIVE DATE'])], np.nan)
df['STATUS REC'] = np.where(df['REC'].isna(), "", np.where(df['Requisition Type'] == "Consignment", "On Time", np.where(df['REC'] >= 1, "Late", "On Time")))
df['ON_TIME'] = np.where(df['STATUS REC'] == 'On Time', 1, np.nan)
df['LATE'] = np.where(df['STATUS REC'] == 'Late', 1, np.nan)
df['ON_TIME%'] = np.where(df['STATUS REC'] == 'On Time', 1, np.where(df['STATUS REC'] == 'Late', 0, np.nan))
special_ontime_categories = ['Jasa/Service', 'Solar']
mask_special_ontime = df['Item Category'].isin(special_ontime_categories)
df.loc[mask_special_ontime, 'STATUS REC'] = 'On Time'
df.loc[mask_special_ontime, 'ON_TIME'] = 1
df.loc[mask_special_ontime, 'LATE'] = np.nan
df.loc[mask_special_ontime, 'ON_TIME%'] = 1
df['ON_TIME%_original_purchasing'] = df['ON_TIME%']
df['LOGISTICAL_PROCESS'] = (df['Final Destination Location'] != df['PO Receive Location']).astype(int)
df['RECEIVE_INDICATOR_PO'] = (df['Qty Order'] == df['Qty Received']).astype(int)
df['RECEIVE_PO_STATUS'] = np.where(df['Qty Received'] == 0, "PO Not Received", np.where(df['Qty Order'] == df['Qty Received'], "Fully Received", "Partial Received"))
df['TL_NUMBER_?'] = df['TL Number'].notnull().astype(int)
df['RECEIVE_INDICATOR_LOGISTIC'] = np.where(
    df['LOGISTICAL_PROCESS'] == 1,
    ((df['TL Qty Received'] == df['Qty Shipped']) & (df['Qty Order'] == df['Qty Received'])).astype(int),
    0
)
conditions_tl = [
    df['LOGISTICAL_PROCESS'] == 0,
    (df['LOGISTICAL_PROCESS'] == 1) & (df['RECEIVE_PO_STATUS'] == "PO Not Received"),
    (df['TL Qty Received'] == 0) & (df['TL Number'].isnull()),
    (df['TL Qty Received'] == 0) & (df['TL Number'].notnull()),
    (df['TL Qty Received'] > 0) & (df['Location TL Received'] != df['Final Destination Location']),
    (df['Location TL Received'] == df['Final Destination Location']) & (df['TL Qty Received'] == df['Qty Shipped']) & (df['Qty Shipped'] == df['Qty Order']) & (df['Qty Order'] == df['Qty Received']),
    (df['Location TL Received'] == df['Final Destination Location'])
]
choices_tl = ["Without Logistical Process", "PO Not Received", "Transfer List Preparation", "On Transit", "At Intermediate Location", "Fully Received", "Partial Received"]
df['TL_RECEIVE_INFO'] = np.select(conditions_tl, choices_tl, default="Check Status")
fully_received_cond = (df['Requisition Type'] == "Consignment") | (df['Item Category'] == "Jasa Logistik") | ((df['LOGISTICAL_PROCESS'] == 0) & (df['RECEIVE_INDICATOR_PO'] == 1)) | ((df['TL_RECEIVE_INFO'] == "Fully Received") & (df['RECEIVE_INDICATOR_PO'] == 1))
df['FULLY_RECEIVE_INFO'] = np.where(fully_received_cond, 1, 0)
df['RECEIVED'] = np.where(df['FULLY_RECEIVE_INFO'] == 1, 1, np.nan)
df['NOT_RECEIVED'] = np.where(df['FULLY_RECEIVE_INFO'] == 0, 1, np.nan)
df['TRANSFER_ITEM'] = np.where((df['LOGISTICAL_PROCESS'] == 0) & (df['TL_NUMBER_?'] == 1), "Transfer Item", "")
df['SHIPPING_TYPE_LAND'] = np.where(df['Shipping Type'].astype(str).str.contains('darat', case=False, na=False), 'Land', '')
df['SHIPPING_TYPE_SEA'] = np.where(df['Shipping Type'].astype(str).str.contains('laut', case=False, na=False), 'Sea', '')
df['SHIPPING_TYPE_AIR'] = np.where(df['Shipping Type'].astype(str).str.contains('udara', case=False, na=False), 'Air', '')


# --- Step 4: Fully Received PO Status ---
print("Running Step 4: Fully Received PO Status...")
po_group_counts = df.groupby('PO Number')['PO Number'].transform('count')
po_group_received = df.groupby('PO Number')['RECEIVED'].transform('sum')
is_fully_received = (po_group_counts == po_group_received)
df['PO_RECEIVE'] = np.where(is_fully_received, 'Fully Received', '')


# --- Step 5: Jasa Service Merge ---
print("Running Step 5: Jasa Service Merge...")
df_itemID_clean = df['Item ID'].astype(str).str.replace(r'\.0$', '', regex=True).str.strip()
df_poNum_clean = df['PO Number'].astype(str).str.replace(r'\.0$', '', regex=True).str.strip()
js_itemID_clean = js['Item ID'].astype(str).str.replace(r'\.0$', '', regex=True).str.strip()
js_poNum_clean = js['PO Number'].astype(str).str.replace(r'\.0$', '', regex=True).str.strip()
df['uid'] = df_itemID_clean + df_poNum_clean
js['uid'] = js_itemID_clean + js_poNum_clean
df = df.merge(js[['uid', 'JS_SERVICE']], on='uid', how='left')
df.drop(columns=['uid'], inplace=True)


# --- Step 6a: Apply Purchasing Business Rule Normalizations ---
print("Running Step 6a: Apply Purchasing Business Rule Normalizations...")
is_jasa_logistik_tracked = (df['Item Category'].str.contains('Jasa Logistik|Solar', na=False)) & (df['VALUE'] == 1)
df.loc[is_jasa_logistik_tracked, 'STATUS_Purchasing'] = 'On Time'
df.loc[is_jasa_logistik_tracked, 'ON_TIME_Purchasing'] = 1
df.loc[is_jasa_logistik_tracked, 'LATE_Purchasing'] = np.nan
df.loc[is_jasa_logistik_tracked, 'ON_TIME%_Purchasing'] = 1


# --- Step 6b: Apply Manual Delivery On-Time Normalizations ---
print("Running Step 6b: Applying Manual Delivery On-Time Normalizations...")
df['PO Number'] = df['PO Number'].astype(str).str.strip()
datanormal['PO Number'] = datanormal['PO Number'].astype(str).str.strip()
datnotcounted['PO Number'] = datnotcounted['PO Number'].astype(str).str.strip()
mask_ontime = df['PO Number'].isin(datanormal['PO Number']) & (df['VALUE'] == 1)
df.loc[mask_ontime, 'STATUS REC'] = 'On Time'
df.loc[mask_ontime, 'ON_TIME'] = 1
df.loc[mask_ontime, 'LATE'] = np.nan
df.loc[mask_ontime, 'ON_TIME%'] = 1
mask_excluded = df['PO Number'].isin(datnotcounted['PO Number'])
df.loc[mask_excluded, 'STATUS REC'] = None
df.loc[mask_excluded, ['ON_TIME', 'LATE', 'ON_TIME%']] = np.nan


# --- Step 6c: Apply Logistic On-Time Normalizations ---
print("Running Step 6c: Applying Logistic On-Time Normalizations...")
logistic_normalized['PO Number'] = logistic_normalized['PO Number'].astype(str).str.strip()
logistic_norm_dict = logistic_normalized.drop_duplicates(subset='PO Number').set_index('PO Number')['Value']
df['ON_TIME%_logistic'] = df['PO Number'].map(logistic_norm_dict)


# --- Step 7: Apply Cost Saving Updates ---
print("Running Step 7: Apply Cost Saving Updates...")
df['Unicode_Key'] = df['Item Name'].astype(str) + '-' + df['PO Number'].astype(str)
cost_saving_df['Unicode_Key'] = cost_saving_df['Item Name'].astype(str) + '-' + cost_saving_df['PO Number'].astype(str)
cost_saving_dict = cost_saving_df.drop_duplicates(subset='Unicode_Key').set_index('Unicode_Key')['Cost Saving']
updated_cost_saving = df['Unicode_Key'].map(cost_saving_dict)
df['Cost Saving'] = updated_cost_saving.fillna(df['Cost Saving'])

# --- Step 8: Create Final_ItemID using ID Library ---
print("Creating Final_ItemID using ID library...")

# 1. Prepare the lookup map
# Convert to numeric, coerce errors → NaN if not convertible
idlibrary_df['Master_ItemID'] = pd.to_numeric(idlibrary_df['Master_ItemID'], errors='coerce').astype('Int64')
idlibrary_df['Variation'] = idlibrary_df['Variation'].astype(str)

item_id_map = {}
for _, row in idlibrary_df.iterrows():
    master_id = row['Master_ItemID']
    # Skip if Master_ItemID is missing
    if pd.isna(master_id):
        continue
    variations = [v.strip() for v in row['Variation'].split(',') if v.strip() != ""]
    for var_id in variations:
        # Only map numeric variations
        try:
            item_id_map[int(var_id)] = int(master_id)
        except ValueError:
            pass

# 2. Apply the mapping to the main DataFrame
# Convert Item ID to numeric, empty/non-numeric → NaN
df['Item ID'] = pd.to_numeric(df['Item ID'], errors='coerce').astype('Int64')

# Map using the dictionary, fallback to NaN if not found
df['Master_ItemID'] = df['Item ID'].map(item_id_map).astype('Int64')

# Map the master IDs, then use .fillna() to keep the original ID if no match is found
#df['Final_ItemID'] = df_item_id_str.map(item_id_map).fillna(df_item_id_str)
# --- END OF NEW LOGIC ---

# --- Step 8: Create Final_ItemID using ID Library ---
print("Applying Routine Categorization Updates...")

category = df['CATEGORYMERGED'].str.lower().fillna('')
item_name = df['Item Name'].str.lower().fillna('')
pic_name = df['Procurement Name'].str.strip().fillna('')

# --- Initialize from existing Routine column ---
df['_Routine'] = df['Routine']

# --- Category-based Rules ---

## 1. Aksesoris Kendaraan → Only "Lampu Rotary" = Routine
df.loc[
    (category == 'aksesoris kendaraan') &
    (item_name.str.contains('lampu rotary', na=False)),
    '_Routine'
] = 'Routine'

## 2. Alat Hiburan → Only "Shuttlecock" = Routine
df.loc[
    (category == 'alat hiburan') &
    (item_name.str.contains('shuttlecock|cock', na=False)),
    '_Routine'
] = 'Routine'

## 3. APD → Specific safety items = Routine
df.loc[
    (category == 'apd') &
    (item_name.str.contains('helm|kacamata|kaca mata|rompi|masker medis|safety shoes|tali|backsupport', na=False)),
    '_Routine'
] = 'Routine'

## 4. Cetak → Always Non-Routine
df.loc[category == 'cetak', '_Routine'] = 'Non-Routine'

## 5. Container & Part → Always Non-Routine
df.loc[category == 'container & part', '_Routine'] = 'Non-Routine'

## 6. Elektrikal → Non-Routine for specific PICs
nonroutine_pics = [
    'rizal agus fianto', 'syifa alifia', 'syifa ramadhani luthfi',
    'linda permata sari', 'puji astuti', 'laurensius adi', 'stheven immanuel'
]
df.loc[
    (category == 'elektrikal') &
    (pic_name.isin(nonroutine_pics)),
    '_Routine'
] = 'Non-Routine'

## 7. Karoseri FT → "Filter" = Routine, else Non-Routine
df.loc[
    (category == 'karoseri ft') &
    (item_name.str.contains('filter', na=False)),
    '_Routine'
] = 'Routine'
df.loc[
    (category == 'karoseri ft') &
    (~item_name.str.contains('filter', na=False)),
    '_Routine'
] = 'Non-Routine'

## 8. Karoseri LT → Always Non-Routine
df.loc[category == 'karoseri lt', '_Routine'] = 'Non-Routine'

## 9. Peralatan Dapur → Always Non-Routine
df.loc[category == 'peralatan dapur', '_Routine'] = 'Non-Routine'

## 10. Peralatan Shipping → "Terpal" = Routine, else Non-Routine
df.loc[
    (category == 'peralatan shipping') &
    (item_name.str.contains('terpal', na=False)),
    '_Routine'
] = 'Routine'
df.loc[
    (category == 'peralatan shipping') &
    (~item_name.str.contains('terpal', na=False)),
    '_Routine'
] = 'Non-Routine'

## 11. Peralatan Survey → "Flagging Tape" = Routine, else Non-Routine
df.loc[
    (category == 'peralatan survey') &
    (item_name.str.contains('flagging tape', na=False)),
    '_Routine'
] = 'Routine'
df.loc[
    (category == 'peralatan survey') &
    (~item_name.str.contains('flagging tape', na=False)),
    '_Routine'
] = 'Non-Routine'

## 12. Telepon → Always Non-Routine
df.loc[category == 'telepon', '_Routine'] = 'Non-Routine'

## 13. Tire DT → Always Routine
df.loc[category == 'tire dt', '_Routine'] = 'Routine'

## 14. Tire Innova → Only "Delium" = Routine
df.loc[
    (category == 'tire innova') &
    (item_name.str.contains('delium', na=False)),
    '_Routine'
] = 'Routine'

## 15. Tire Manhaul → "GT" or "Gajah Tunggal" = Routine
df.loc[
    (category == 'tire manhaul') &
    (item_name.str.contains('gt|gajah tunggal', na=False)),
    '_Routine'
] = 'Routine'

## 16. Tire TL → Always Non-Routine
df.loc[category == 'tire tl', '_Routine'] = 'Non-Routine'

## 17. Tire VB → Always Routine
df.loc[category == 'tire vb', '_Routine'] = 'Routine'

## 18. Radio HT, Rig → Always Non-Routine
df.loc[category == 'radio ht, rig', '_Routine'] = 'Non-Routine'

## 19. Packaging → Always Routine
df.loc[category == 'packaging', '_Routine'] = 'Routine'

## 20. On time unnormalized
print("Pulling unnormalized On time%...")
df['ON_TIME%_overall_original'] = np.where(
    df['Item Category'].isin(['Jasa/Service', 'Solar']),  # IF it's a special category...
    1,                                                    # THEN force it to 1
    np.where(                                             # ELSE, run your original check:
        df['REC'].isna(),                                   # IF REC is NaN...
        np.nan,                                             # THEN set to NaN
        np.where(df['REC'] >= 1, 0, 1)                      # ELSE, check if >= 1 (Late=0) or < 1 (On Time=1)
    )
)

## 21. Update 2024 Logistic Cost Saving
print("Updating 2024 Logistic Cost Saving...")
df['Cost Saving'] = np.where(
    (df['Item ID'] == 18640) & (df['PO Number'] == '20/CB/012024'),
    674957950,
    df['Cost Saving']
)


# --- Final Column Ordering and Cleanup ---
print("Reordering columns and performing final cleanup...")
final_column_order = [
    'Requisition Type', 'Item ID', 'Item Name', 'Item Category', 'Department', 
    'Unit', 'Currency', 'Exchange Rate', 'PO Price', 'Qty Order', 
    'PO Disc/Cost', 'PO Sub Total', 'Jumlah PPN', 'Qty Received', 'PO Receive Location', 
    'PO Submit Date', 'PO Required Date', 'PO Approval Date', 'Receive PO Estimation', 'Receive PO Date', 
    'Qty Handover', 'Handover Date', 'PO Number', 'Supplier', 'Supplier Location', 
    'Term of Payment', 'PO Status', 'PO Progress Status', 'Status Update Date', 'Requisition Number', 
    'Requisition Date', 'Requisition Submited Date', 'Requisition Approved Date', 'Requisition Required Date', 'Qty Requisition', 
    'Requisition Unit Price', 'Requisition SubTotal', 'Asset / Non Asset', 'Cost Saving', 'Routine', 
    'Urgent', 'Background Needs', 'Urgent Note', 'Urgent Cost', 'Procurement Name', 
    'Req Status', 'Req Progress Status', 'TL Number', 'Shipping Type', 'Created TL Date', 
    'Qty Shipped', 'Shipped Date', 'ETA Date', 'TL Qty Received', 'Received TL Date', 
    'Location TL Received', 'Final Destination Location', 'Remarks', 'VALUE', 'UNIQUE COUNT PO', 
    'CATEGORYMERGED', 'LOC', 'LEAD TIME', 'URGENT_NORMAL', 'NORMAL', 
    'URGENT2', 'URGENT*', 'URGENT_FINALFORLOGBOOK', 'WILAYAH', 'PULAU', 
    'DEPARTMENT_', 'DIVISI', 'SUPPLIER_', 'TOP', 'Updated Requisition Approved Date', 
    'Updated Requisition Required Date', 'Background Update', 'TIME DATE', 'PR - PO', 'PO SUB - PO APP', 
    'PO - R PO', 'R-R SITE', 'PR - PO SUB WD', 'RPO-TLC', 'TLC-SHIP', 
    'SHIP-RSITE', 'REQUISITION_TOTAL', 'PO_TOTAL', 'BUDGET', 'BUDGET%', 
    'FARTHEST REQUIRED DATE', 'USED RECEIVE DATE', 'REC', 'STATUS REC', 'ON_TIME', 
    'LATE', 'ON_TIME%', 'LOGISTICAL_PROCESS', 'RECEIVE_INDICATOR_PO', 'RECEIVE_PO_STATUS', 
    'TL_NUMBER_?', 'RECEIVE_INDICATOR_LOGISTIC', 'TL_RECEIVE_INFO', 'FULLY_RECEIVE_INFO', 'TRANSFER_ITEM', 'SHIPPING_TYPE_LAND', 
    'SHIPPING_TYPE_SEA', 'SHIPPING_TYPE_AIR', 'LOGISTIC_FREIGHT', 'RECEIVED', 'NOT_RECEIVED', 
    'PO_RECEIVE', 'JS_SERVICE', 'ON_TIME%_overall_original' ,'ON_TIME%_original_purchasing', 'ON_TIME%_logistic', 'Final_ItemID',
    'Purchasing_Duration', 'STATUS_Purchasing', 'ON_TIME_Purchasing', 'LATE_Purchasing', 'ON_TIME%_Purchasing','_Routine'
]
for col in final_column_order:
    if col not in df.columns:
        df[col] = np.nan
df = df[final_column_order]
df.drop(columns=['Unicode_Key', 'CATEGORYVALUEXCMG', 'CATEGORYVALUE'], inplace=True, errors='ignore')

print("Processing complete!")

Preparing data and normalizing dates...
Running Step 1: Initial Feature Engineering...
Running Step 2: Time-Based Calculations...
Calculating Purchasing On-Time metric...
Running Step 3: Logistics & Receiving Status...
Running Step 4: Fully Received PO Status...
Running Step 5: Jasa Service Merge...
Running Step 6a: Apply Purchasing Business Rule Normalizations...
Running Step 6b: Applying Manual Delivery On-Time Normalizations...
Running Step 6c: Applying Logistic On-Time Normalizations...
Running Step 7: Apply Cost Saving Updates...
Creating Final_ItemID using ID library...
Applying Routine Categorization Updates...
Pulling unnormalized On time%...
Reordering columns and performing final cleanup...
Processing complete!


In [5]:
### 5. FINAL OUTPUT
print("Final DataFrame columns:")
print(df.columns.tolist())

print("\nFinal DataFrame preview:")
display(df.head())

# --- Save the final combined output to a new Excel file ---
output_path = os.path.join(base_path, 'output', 'Consolidated_Logbook_Output_f2.xlsx')
df.to_excel(output_path, index=False)

print(f"\nSuccessfully saved the final output to: {output_path}")

Final DataFrame columns:
['Requisition Type', 'Item ID', 'Item Name', 'Item Category', 'Department', 'Unit', 'Currency', 'Exchange Rate', 'PO Price', 'Qty Order', 'PO Disc/Cost', 'PO Sub Total', 'Jumlah PPN', 'Qty Received', 'PO Receive Location', 'PO Submit Date', 'PO Required Date', 'PO Approval Date', 'Receive PO Estimation', 'Receive PO Date', 'Qty Handover', 'Handover Date', 'PO Number', 'Supplier', 'Supplier Location', 'Term of Payment', 'PO Status', 'PO Progress Status', 'Status Update Date', 'Requisition Number', 'Requisition Date', 'Requisition Submited Date', 'Requisition Approved Date', 'Requisition Required Date', 'Qty Requisition', 'Requisition Unit Price', 'Requisition SubTotal', 'Asset / Non Asset', 'Cost Saving', 'Routine', 'Urgent', 'Background Needs', 'Urgent Note', 'Urgent Cost', 'Procurement Name', 'Req Status', 'Req Progress Status', 'TL Number', 'Shipping Type', 'Created TL Date', 'Qty Shipped', 'Shipped Date', 'ETA Date', 'TL Qty Received', 'Received TL Date', 'L

,Requisition Type,Item ID,Item Name,Item Category,Department,Unit,Currency,Exchange Rate,PO Price,Qty Order,...,ON_TIME%_overall_original,ON_TIME%_original_purchasing,ON_TIME%_logistic,Final_ItemID,Purchasing_Duration,STATUS_Purchasing,ON_TIME_Purchasing,LATE_Purchasing,ON_TIME%_Purchasing,_Routine
0,General,73471,Face Shield UV400,APD,MMP-PLNT__Seragam_Safety-LAR,Pcs,IDR,1.0,40000.0,10.0,...,NaN,NaN,NaN,73471,11.0,Late,NaN,1.0,0.0,Non-Routine
1,General,60402,4C4782 Oring Kit Oring Box - HG,Tools dan Alat Ukur,BTG-PLNT__Tools_Besi_Cat_Pipa-OBI,box,IDR,1.0,1700000.0,3.0,...,NaN,NaN,NaN,60402,6.0,Late,NaN,1.0,0.0,Routine
2,General,62065,DCD805 D2 Cordless Brushless Hammer Drill 20 V...,Tools dan Alat Ukur,BTG-PLNT__Tools_Besi_Cat_Pipa-OBI,pcs,IDR,1.0,3500000.0,2.0,...,NaN,NaN,NaN,62065,7.0,Late,NaN,1.0,0.0,Routine
3,General,52621,Digital Delay Timer Trigger On-Off Mosfet 5-36...,Spare Part EXC 370CA XCMG,BTG-PLNT__A2B_Accu-OBI,Pcs,IDR,1.0,37500.0,20.0,...,NaN,NaN,NaN,52621,3.0,On Time,1.0,NaN,1.0,Routine
4,General,73814,Banner Internal Ukuran Panjang 3 Meter x Lebar...,Cetak,MMP-OPS_HSE_Enviro_Cetak-OBI,Lembar,IDR,1.0,69000.0,1.0,...,NaN,NaN,NaN,73814,2.0,On Time,1.0,NaN,1.0,Non-Routine



Successfully saved the final output to: C:\Users\user\Documents\PROC\Monthly - Weekly\2025\11November\11November Week 2 1011\output\Consolidated_Logbook_Output_f2.xlsx
